In [5]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [7]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [34]:
!pip install pyttsx3


## Import libraries

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import cv2
import numpy as np
from tqdm import tqdm
import pyttsx3

## Define class labels for the sign language dataset

In [56]:
class_labels = ['Hello', 'IloveYou', 'No', 'Please', 'Thanks', 'Yes']

## Define a custom PyTorch dataset for loading images and labels

In [58]:
# Define dataset class
class SignLanguageDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))
        self.label_files = sorted(os.listdir(label_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        label_path = os.path.join(self.label_dir, self.label_files[idx])
        
        image = Image.open(img_path).convert("RGB")
        
        # Read YOLO format label (only the first value: class ID)
        with open(label_path, "r") as file:
            label = int(file.readline().split()[0])
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

## Define image transformations for data preprocessing

In [59]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [60]:

# # Data transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

##  Load training and testing datasets

In [61]:
# Dataset paths
train_img_path = r"C:\Users\inGnious AI\Desktop\My\sign_data\train\images"
train_label_path = r"C:\Users\inGnious AI\Desktop\My\sign_data\train\labels"
test_img_path = r"C:\Users\inGnious AI\Desktop\My\sign_data\test\images"
test_label_path = r"C:\Users\inGnious AI\Desktop\My\sign_data\test\labels"

train_dataset = SignLanguageDataset(train_img_path , train_label_path, transform=transform)
test_dataset = SignLanguageDataset(test_img_path, test_label_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Define a PyTorch model using ResNet18 as a base model

In [62]:
# Define model
class SignLanguageModel(nn.Module):
    def __init__(self, num_classes=6):
        super(SignLanguageModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)
        
    def forward(self, x):
        return self.base_model(x)
model = SignLanguageModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SignLanguageModel(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

## Define loss function and optimizer

In [63]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Function to train the model

In [64]:
# Training loop
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

## Function to evaluate the model on test data

In [65]:
# Testing loop
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")

## Train and test the model

In [66]:
# Train and test the model
train(model, train_loader, criterion, optimizer, epochs=35)
test(model, test_loader)


00%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.02s/it]

Epoch 1, Loss: 1.5762709081172943



00%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.71s/it]

Epoch 2, Loss: 0.9164660274982452



00%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.57s/it]

...

Epoch 33, Loss: 0.0023199709539767355



00%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.33s/it]

Epoch 34, Loss: 0.001455498655559495



00%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.44s/it]

Epoch 35, Loss: 0.001595219218870625
Test Accuracy: 100.0%


## Save the trained model

In [67]:
torch.save(model.state_dict(), "sign_language_model.pth")
print("Model saved successfully!")

Model saved successfully!


## Function for real-time sign language recognition using webcam

In [68]:
# Initialize text-to-speech engine
engine = pyttsx3.init()

def speak(text):
    engine.say(text)
    engine.runAndWait()

## Load the trained model for inference

In [69]:
model.load_state_dict(torch.load("sign_language_model.pth"))
model.eval()
print("Model loaded successfully!")

Model loaded successfully!


## Real-time inference using webcam

In [70]:
def live_prediction(model):
    cap = cv2.VideoCapture(0)
    model.eval()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
        label = class_labels[predicted.item()]
        cv2.putText(frame, f"Prediction: {label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Sign Language Recognition', frame)
        
        # Convert label to speech
        speak(label)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [71]:
#  to run live webcam detection
live_prediction(model)